## Programando em CUDA - Aula 07

# Comandos básicos

Código para a execução de programas CUDA no ambiente Colab

In [ ]:
!pip install nvcc4jupyter

In [ ]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpyjb09l2r".


Descobrindo qual GPU foi alocada

In [ ]:
!nvidia-smi

Wed Apr  2 14:04:50 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   65C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Verificando a versão do nvcc

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


# Identificadores locais e globais de threads

In [ ]:
%%cuda
#include <stdio.h>

__global__ void loop()
{
  /*
   * This idiomatic expression gives each thread
   * a unique index within the entire grid.
   */

  int i = blockIdx.x * blockDim.x + threadIdx.x;
  printf("%d\n", i);
}

int main()
{
  /*
   * Additional execution configurations that would
   * work and meet the exercises contraints are:
   *
   */

  loop<<<10, 3>>>();
  cudaDeviceSynchronize();
}

# Impressão de algumas características da placa de vídeo

In [ ]:
%%cuda
#include <stdio.h>


// Print device properties
void printDevProp(cudaDeviceProp devProp)
{
    printf("Major revision number:         %d\n",  devProp.major);
    printf("Minor revision number:         %d\n",  devProp.minor);
    printf("Name:                          %s\n",  devProp.name);
    printf("Total global memory:           %lu\n",  devProp.totalGlobalMem);
    printf("Total shared memory per block: %lu\n",  devProp.sharedMemPerBlock);
    printf("Total registers per block:     %d\n",  devProp.regsPerBlock);
    printf("Warp size:                     %d\n",  devProp.warpSize);
    printf("Maximum memory pitch:          %lu\n",  devProp.memPitch);
    printf("Maximum threads per block:     %d\n",  devProp.maxThreadsPerBlock);
    for (int i = 0; i < 3; ++i)
        printf("Maximum dimension %d of block:  %d\n", i, devProp.maxThreadsDim[i]);
    for (int i = 0; i < 3; ++i)
        printf("Maximum dimension %d of grid:   %d\n", i, devProp.maxGridSize[i]);
    printf("Clock rate:                    %d\n",  devProp.clockRate);
    printf("Total constant memory:         %lu\n",  devProp.totalConstMem);
    printf("Texture alignment:             %lu\n",  devProp.textureAlignment);
    printf("Concurrent copy and execution: %s\n",  (devProp.deviceOverlap ? "Yes" : "No"));
    printf("Number of multiprocessors:     %d\n",  devProp.multiProcessorCount);
    printf("Kernel execution timeout:      %s\n",  (devProp.kernelExecTimeoutEnabled ?"Yes" : "No"));
    return;
}

int main()
{
    int devCount;
    cudaGetDeviceCount(&devCount);
    printf("CUDA Device Query...\n");
    printf("There are %d CUDA devices.\n", devCount);

    for (int i = 0; i < devCount; ++i)
    {
        // Get device properties
        printf("\nCUDA Device #%d\n", i);
        cudaDeviceProp devProp;
        cudaGetDeviceProperties(&devProp, i);
        printDevProp(devProp);
    }
    return 0;
}

CUDA Device Query...
There are 1 CUDA devices.

CUDA Device #0
Major revision number:         7
Minor revision number:         5
Name:                          Tesla T4
Total global memory:           15828320256
Total shared memory per block: 49152
Total registers per block:     65536
Warp size:                     32
Maximum memory pitch:          2147483647
Maximum threads per block:     1024
Maximum dimension 0 of block:  1024
Maximum dimension 1 of block:  1024
Maximum dimension 2 of block:  64
Maximum dimension 0 of grid:   2147483647
Maximum dimension 1 of grid:   65535
Maximum dimension 2 of grid:   65535
Clock rate:                    1590000
Total constant memory:         65536
Texture alignment:             512
Concurrent copy and execution: Yes
Number of multiprocessors:     40
Kernel execution timeout:      No



# Questão 1

Crie um código que imprima os valores pares de 0 até 20 utilizando programação CUDA

In [ ]:
%%cuda
#include <stdio.h>

__global__ void num_pares(){
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx%2==0){
      printf("%d \n",idx);
    }
}

int main(){

    num_pares<<<2,10>>>();
    cudaDeviceSynchronize();
    return 0;
}

# Evitando acessos fora da faixa do vetor

In [ ]:
%%cuda
#include <stdio.h>


__global__ void initializeElementsTo(int initialValue, int *a, int N)
{
  int i = threadIdx.x + blockIdx.x * blockDim.x;
  if (i < N)
  {
    a[i] = initialValue;
  }
}

int main()
{
  /*
   * Do not modify `N`.
   */

  int N = 1000;

  int *a;
  size_t size = N * sizeof(int);

  cudaMallocManaged(&a, size);

  /*
   * Assume we have reason to want the number of threads
   * fixed at `256`: do not modify `threads_per_block`.
   */

  size_t threads_per_block = 256;

  /*
   * The following is idiomatic CUDA to make sure there are at
   * least as many threads in the grid as there are `N` elements.
   */

  size_t number_of_blocks = 4;
  //size_t number_of_blocks = (N + threads_per_block - 1) / threads_per_block;

  int initialValue = 6;

  initializeElementsTo<<<number_of_blocks, threads_per_block>>>(initialValue, a, N);
  cudaDeviceSynchronize();

  /*
   * Check to make sure all values in `a`, were initialized.
   */

  for (int i = 0; i < N; ++i)
  {
    if(a[i] != initialValue)
    {
      printf("FAILURE: target value: %d\t a[%d]: %d\n", initialValue, i, a[i]);
      cudaFree(a);
      exit(1);
    }
  }
  printf("SUCCESS!\n");

  cudaFree(a);
}

FAILURE: target value: 6	 a[0]: 0



# Lidando com mais dados do que threads: strides (passadas)

In [ ]:
%%cuda
#Código removido


All elements were doubled? TRUE



Exemplo utilizando %%writefile e compilando o arquivo

In [ ]:
%%writefile exemplo.cu

#Código removido

Writing exemplo.cu


In [ ]:
!nvcc -o exemplo exemplo.cu

In [ ]:
!./exemplo

All elements were doubled? TRUE


# Questão 2

Considerando os exemplos acima, crie um vetor de 100 posições e inicialize-o de forma parelela com valor 100. Em seguida, incremente-o em ordem crescente (de 0 à 99) em cada posição do vetor (da posição 0 à 99). Confira se seu resultado está correto.

In [ ]:
%%cuda
#include <stdio.h>
#include <cuda_runtime.h>

//
__global__ void initializeVector(int *vec, int size, int value) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        vec[idx] = value;
    }
}

// Kernel para incrementar cada elemento com seu índice
__global__ void incrementWithIndex(int *vec, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        vec[idx] += idx;
    }
}

// Função para verificar o resultado
bool checkResult(int *vec, int size) {
    for (int i = 0; i < size; i++) {
        if (vec[i] != (100 + i)) {
            printf("Erro na posição %d: esperado %d, encontrado %d\n",
                   i, 100 + i, vec[i]);
            return false;
        }
    }
    return true;
}

int main() {
    const int N = 100;
    int *vec;
    size_t size = N * sizeof(int);

    // Alocar memória gerenciada
    cudaMallocManaged(&vec, size);

    // Configurar a execução dos kernels
    int threadsPerBlock = 32;
    int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock;

    // Inicializar o vetor com 100
    initializeVector<<<blocksPerGrid, threadsPerBlock>>>(vec, N, 100);
    cudaDeviceSynchronize();

    // Incrementar cada posição com seu índice
    incrementWithIndex<<<blocksPerGrid, threadsPerBlock>>>(vec, N);
    cudaDeviceSynchronize();

    // Verificar o resultado
    if (checkResult(vec, N)) {
        printf("Resultado correto! Todos os elementos foram processados adequadamente.\n");

        // Opcional: imprimir os primeiros e últimos elementos para verificação
        printf("Primeiros 5 elementos: ");
        for (int i = 0; i < 5; i++) printf("%d ", vec[i]);
        printf("\nÚltimos 5 elementos: ");
        for (int i = N-5; i < N; i++) printf("%d ", vec[i]);
        printf("\n");
    }

    // Liberar memória
    cudaFree(vec);

    return 0;
}

Erro na posição 0: esperado 100, encontrado 0



# Questão 3

 Considerando os exemplos acima, crie dois vetores de 1000 posições e inicialize o primeiro vetor de forma parelela com valor decrescente (de 0 à 999) e o segundo vetor com valores crescentes (de 0 à 99). Em seguida, faça a soma dos dois vetores, elemento a elemento. Crie uma função sequencial para retornar se todos os valores do vetor resultante são iguais.

In [ ]:
#Código da questão 3

# Programa para somar dois vetores

In [ ]:
%%writefile teste.cu
#Código removido

Writing teste.cu


In [ ]:
!nvcc -o teste teste.cu -run

Success! All values calculated correctly.
Success! All values calculated correctly.
Success! All values calculated correctly.


In [ ]:
!nvprof ./teste

==7372== NVPROF is profiling process 7372, command: ./teste
Success! All values calculated correctly.
Success! All values calculated correctly.
Success! All values calculated correctly.
==7372== Profiling application: ./teste
==7372== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  3.85746s         2  1.92873s  1.81797s  2.03949s  addVectorsInto(float*, float*, float*, int)
      API calls:   96.89%  3.85747s         1  3.85747s  3.85747s  3.85747s  cudaDeviceSynchronize
                    2.57%  102.34ms         3  34.112ms  25.675us  102.24ms  cudaMallocManaged
                    0.53%  20.951ms         3  6.9836ms  6.9513ms  7.0338ms  cudaFree
                    0.01%  322.73us         2  161.36us  7.5560us  315.17us  cudaLaunchKernel
                    0.00%  148.76us       114  1.3040us     137ns  55.222us  cuDeviceGetAttribute
                    0.00%  29.455us         1  29.455us  29.455us  29.45

#Questão 4


Implemente um código para inicializar dois vetores de tamanho N ($N=70$). O primeiro vetor deve ser inicializado com valor igual a 10 e o segundo vetor deve ser inicializado com valor crescente (de 0 a $N-1$). Imprima a multiplicação do primeiro vetor com o segundo vetor, elemento a elemento. Obs.: Considere o bloco com 32 threads.

In [ ]:
%%cuda
#include <stdio.h>
#include <cuda_runtime.h>

// Kernel para inicializar o vetor com valor constante
__global__ void initializeConstant(int *vec, int size, int value) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        vec[idx] = value;
    }
}

// Kernel para inicializar o vetor com valores crescentes
__global__ void initializeIncremental(int *vec, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        vec[idx] = idx;
    }
}

// Kernel para multiplicar os vetores elemento a elemento
__global__ void multiplyVectors(int *vecA, int *vecB, int *result, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        result[idx] = vecA[idx] * vecB[idx];
    }
}

int main() {
    const int N = 70;
    int *vecA, *vecB, *result;
    size_t size = N * sizeof(int);

    // Alocar memória gerenciada para os vetores
    cudaMallocManaged(&vecA, size);
    cudaMallocManaged(&vecB, size);
    cudaMallocManaged(&result, size);

    // Configurar a execução dos kernels (32 threads por bloco)
    int threadsPerBlock = 32;
    int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock;

    // Inicializar os vetores
    initializeConstant<<<blocksPerGrid, threadsPerBlock>>>(vecA, N, 10);
    initializeIncremental<<<blocksPerGrid, threadsPerBlock>>>(vecB, N);
    cudaDeviceSynchronize();

    // Multiplicar os vetores
    multiplyVectors<<<blocksPerGrid, threadsPerBlock>>>(vecA, vecB, result, N);
    cudaDeviceSynchronize();

    // Imprimir os resultados
    printf("Multiplicação dos vetores (elemento a elemento):\n");
    for (int i = 0; i < N; i++) {
        printf("%d * %d = %d\n", vecA[i], vecB[i], result[i]);
    }

    // Liberar memória
    cudaFree(vecA);
    cudaFree(vecB);
    cudaFree(result);

    return 0;
}

Multiplicação dos vetores (elemento a elemento):
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0
0 * 0 = 0



#Questão 5

Implemente um código para inicializar um vetor com 1000 posições com valores entre 0 a 999. Eleve todos os valores ao quadrado ($x^2$). Retorne se as operações foram realizadas com sucesso ou se houve falha. Obs.: número de blocos = 32 e número de threads igual a 256 por bloco.

In [ ]:
%%cuda
#include <stdio.h>
#include <cuda_runtime.h>

// Kernel para inicialização do vetor com valores de 0 a 999
__global__ void initializeVector(int *vec, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        vec[idx] = idx;
    }
}

// Kernel para elevar os elementos ao quadrado
__global__ void squareVector(int *vec, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        vec[idx] = vec[idx] * vec[idx];
    }
}

// Função de verificação
bool verifyResults(int *vec, int size) {
    for (int i = 0; i < size; i++) {
        if (vec[i] != i * i) {
            printf("Erro na posição %d: Esperado %d, Obtido %d\n", i, i*i, vec[i]);
            return false;
        }
    }
    return true;
}

int main() {
    const int N = 1000;
    int *vec;
    size_t size = N * sizeof(int);
    cudaError_t err;

    // Alocar memória gerenciada
    err = cudaMallocManaged(&vec, size);
    if (err != cudaSuccess) {
        printf("Erro na alocação: %s\n", cudaGetErrorString(err));
        return 1;
    }

    // Configuração da execução
    int threadsPerBlock = 256;
    int blocksPerGrid = 32;

    // Inicializar vetor
    initializeVector<<<blocksPerGrid, threadsPerBlock>>>(vec, N);
    err = cudaDeviceSynchronize();
    if (err != cudaSuccess) {
        printf("Erro na inicialização: %s\n", cudaGetErrorString(err));
        cudaFree(vec);
        return 1;
    }

    // Elevar ao quadrado
    squareVector<<<blocksPerGrid, threadsPerBlock>>>(vec, N);
    err = cudaDeviceSynchronize();
    if (err != cudaSuccess) {
        printf("Erro na operação de quadrado: %s\n", cudaGetErrorString(err));
        cudaFree(vec);
        return 1;
    }

    // Verificar resultados
    if (verifyResults(vec, N)) {
        printf("Sucesso! Todos os elementos foram processados corretamente.\n");
    } else {
        printf("Falha! Erros encontrados nos resultados.\n");
    }

    // Liberar memória
    cudaFree(vec);
    return 0;
}

Erro na posição 1: Esperado 1, Obtido 0
Falha! Erros encontrados nos resultados.



#Questão 6

Proponha uma solução paralela em CUDA para o cálculo da seguinte expressão vetorial: $V=k_1*A+k_2*B$, onde $k_1$ e $k_2$ são constantes e $A$ e $B$ são vetores de tamanho $n=500$. Utilize $n_{blocos}=8$ e $n_{threads}=16$. Inicialize os vetores $A$ e $B$ de forma paralela e com valores arbitrários (definidos pelo programador). Imprima o vetor $V$.

In [ ]:
#Insira seu código aqui

#Questão 7

Implemente um código para inicializar dois vetores de tamanho N ($N=1000$). O primeiro vetor deve ser inicializado com valor crescente (de 0 a $N-1$) e o segundo vetor deve ser inicializado com valor decrescente (de $N-1$ a 0). Realize a soma do primeiro vetor com o segundo vetor, elemento a elemento, e insira em um terceiro vetor. Verifique se todos os valores são iguais, caso positivo, imprima “Sucesso! Todos os valores sao identicos”, senão, imprima “Erro detectado no codigo”. Obs.:  número de blocos = 8 e número de threads igual a 16 por bloco.

In [ ]:
#Insira seu código aqui

#Utilize a seguinte função de verificação
void verificacao(int *c, int N)
{
  for(int i = 0; i < N; i++)
  {
    if(c[i] != c[0])
    {
      printf("Erro detectado no codigo.\n");
      exit(1);
    }
  }
  printf("Sucesso! Todos os valores sao identicos.\n");
}
